In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, current_date, max, min, year, split, concat, lit, floor, when
spark = SparkSession.builder.appName("Leaguepedia").config("spark.jars", "/home/quocthanh/spark/jars/postgresql-42.7.4.jar").getOrCreate()


your 131072x1 screen size is bogus. expect trouble
24/10/14 19:36:22 WARN Utils: Your hostname, quocthanh resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/14 19:36:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/10/14 19:36:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Q1: List top 5 countries with the most players.

In [3]:
players_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://172.26.115.201:5432/ETL_Leaguepedia") \
    .option("dbtable", "players") \
    .option("user", "quocthanh") \
    .option("password", "quocthanh1804") \
    .load()


In [4]:
main_players = players_df.filter(col("role").isin(["Top", "Jungle", "Mid", "Bot", "Support"]))

In [5]:

top5_countries_players = main_players.groupBy("country").count().orderBy(col("count").desc())
top5_countries = top5_countries_players.limit(5).toPandas()["country"].tolist()
top5_countries_players.show()

+--------------+-----+
|       country|count|
+--------------+-----+
| United States|  876|
|         China|  828|
|   South Korea|  798|
|        Brazil|  339|
|         Spain|  330|
|        Turkey|  322|
|       Germany|  317|
|        Poland|  293|
|        Taiwan|  270|
|     Argentina|  270|
|        Canada|  267|
|        France|  259|
|Czech Republic|  256|
|         Chile|  251|
|       Vietnam|  247|
|          NULL|  217|
|        Mexico|  212|
|        Greece|  209|
|   Netherlands|  186|
|      Portugal|  184|
+--------------+-----+
only showing top 20 rows



In [6]:
players_per_role = main_players.filter(col("country").isin(top5_countries)).groupBy("country", "role").count().orderBy("country", "role")
players_per_role.show()

+-----------+-------+-----+
|    country|   role|count|
+-----------+-------+-----+
|     Brazil|    Bot|   77|
|     Brazil| Jungle|   67|
|     Brazil|    Mid|   64|
|     Brazil|Support|   62|
|     Brazil|    Top|   69|
|      China|    Bot|  161|
|      China| Jungle|  199|
|      China|    Mid|  140|
|      China|Support|  164|
|      China|    Top|  164|
|South Korea|    Bot|  158|
|South Korea| Jungle|  151|
|South Korea|    Mid|  161|
|South Korea|Support|  174|
|South Korea|    Top|  154|
|      Spain|    Bot|   68|
|      Spain| Jungle|   73|
|      Spain|    Mid|   61|
|      Spain|Support|   61|
|      Spain|    Top|   67|
+-----------+-------+-----+
only showing top 20 rows



Q2: Oldest and youngest active players

In [7]:
main_df_q2 = main_players.filter((col("isretired") == 0) & (col("team").isNotNull()) & (col("birthdate").isNotNull())).select("player", "country", "birthdate", "team", "role")
main_df_q2 = main_df_q2.withColumn("age", datediff(current_date(), col("birthdate")))
oldest_player = main_df_q2.orderBy(col("age").desc()).limit(1)
youngest_player = main_df_q2.orderBy(col("age")).limit(1)
oldest_player.show()
youngest_player.show()

+------+-------+----------+------------+----+-----+
|player|country| birthdate|        team|role|  age|
+------+-------+----------+------------+----+-----+
|Broeki|Germany|1991-02-11|No Need Orga| Bot|12299|
+------+-------+----------+------------+----+-----+

+--------+-----------+----------+--------------------+-------+----+
|  player|    country| birthdate|                team|   role| age|
+--------+-----------+----------+--------------------+-------+----+
|Antimage|South Korea|2008-12-07|Hanwha Life Espor...|Support|5790|
+--------+-----------+----------+--------------------+-------+----+



Q3: Which champions are mostly played by each role ?

In [8]:
players_favchamps = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://172.26.115.201:5432/ETL_Leaguepedia") \
    .option("dbtable", "playersfavchamps") \
    .option("user", "quocthanh") \
    .option("password", "quocthanh1804") \
    .load()

In [9]:
q2_main_df = players_favchamps.join(main_players, "player").select("player", "role", "champ")

In [10]:
#Top most popular champions
q2_top = q2_main_df.filter(col("role") == "Top").groupBy("champ").count().orderBy(col("count").desc()).limit(5).show()
q2_jungle = q2_main_df.filter(col("role") == "Jungle").groupBy("champ").count().orderBy(col("count").desc()).limit(5).show()
q2_mid = q2_main_df.filter(col("role") == "Mid").groupBy("champ").count().orderBy(col("count").desc()).limit(5).show()
q2_bot = q2_main_df.filter(col("role") == "Bot").groupBy("champ").count().orderBy(col("count").desc()).limit(5).show()
q2_support = q2_main_df.filter(col("role") == "Support").groupBy("champ").count().orderBy(col("count").desc()).limit(5).show()

+-------+-----+
|  champ|count|
+-------+-----+
|Camille|  132|
| Irelia|   99|
|  Jayce|   78|
| Aatrox|   73|
|    Jax|   71|
+-------+-----+

+-------+-----+
|  champ|count|
+-------+-----+
|Lee Sin|  246|
| Graves|  132|
|Nidalee|  106|
|Kha'Zix|  103|
|  Elise|   87|
+-------+-----+

+-------+-----+
|  champ|count|
+-------+-----+
|LeBlanc|  105|
|    Zed|   87|
|Orianna|   87|
|   Ahri|   81|
|  Akali|   79|
+-------+-----+

+-------+-----+
|  champ|count|
+-------+-----+
| Ezreal|  198|
| Kai'Sa|  193|
|  Vayne|  147|
| Lucian|  137|
|Caitlyn|   95|
+-------+-----+

+--------+-----+
|   champ|count|
+--------+-----+
|  Thresh|  264|
|   Rakan|  134|
|Nautilus|   89|
|    Bard|   85|
| Alistar|   78|
+--------+-----+



In [11]:
scoreboardgames = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://quocthanh:5432/ETL_Leaguepedia") \
    .option("dbtable", "scoreboardgames") \
    .option("user", "quocthanh") \
    .option("password", "quocthanh1804") \
    .load()
scoreboardgames = scoreboardgames.withColumn("gamelength_in_seconds", split(col("gamelength"), ":")[0]*60 + split(col("gamelength"), ":")[1])

Q4: Average game duration per year

In [12]:
q4 = scoreboardgames.groupBy(year("timestamp").alias("year")).avg("gamelength_in_seconds").orderBy("year") \
    .withColumn("avg(gamelength_in_seconds)", concat(floor(col("avg(gamelength_in_seconds)").cast('int') / 60), lit(":"), floor(col("avg(gamelength_in_seconds)") % 60)))
q4.show()

+----+--------------------------+
|year|avg(gamelength_in_seconds)|
+----+--------------------------+
|2011|                     37:57|
|2012|                     34:47|
|2013|                     32:54|
|2014|                     35:21|
|2015|                     35:52|
|2016|                     37:23|
|2017|                      36:1|
|2018|                     34:38|
|2019|                      33:2|
|2020|                     32:53|
|2021|                     32:13|
|2022|                     32:25|
|2023|                     31:27|
|2024|                     32:21|
+----+--------------------------+



In [13]:
gamebans = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://quocthanh:5432/ETL_Leaguepedia") \
    .option("dbtable", "gamebans") \
    .option("user", "quocthanh") \
    .option("password", "quocthanh1804") \
    .load()

Q5: Which champion is mostly banned first in Worlds/ MSI ?

In [14]:
from pyspark.sql.types import StringType, StructField, IntegerType, StructType
schema = StructType([
    StructField("league", StringType(), True),
    StructField("champion", StringType(), True),
    StructField("count", IntegerType(), True),
])
league_bans = spark.createDataFrame([], schema)

for i in range(0, 14):
    if i < 3:
        league = f"Season {i + 1} World Championship"
        game = gamebans.filter(col("gameid").contains(league))
        ban = game.filter(col("ban_order") == 0).groupBy("champ").count().orderBy(col("count").desc()).limit(1).collect()
        league_renamed = f"{2011 + i} Season World Championship"
        row = [(league_renamed, ban[0]["champ"], ban[0]["count"])]
        league_bans = league_bans.union(spark.createDataFrame(row, schema))
    else:
        game = gamebans.filter(col("gameid").contains(f"{2011 + i} Season World Championship"))
        ban = game.filter(col("ban_order") == 0).groupBy("champ").count().orderBy(col("count").desc()).limit(1).collect()
        row = [(f"{2011 + i} Season World Championship", ban[0]["champ"], ban[0]["count"])]
        league_bans = league_bans.union(spark.createDataFrame(row, schema))

for i in range(0, 10):
    league = f"{2015 + i} Mid-Season Invitational"
    game = gamebans.filter(col("gameid").contains(league))
    ban = game.filter(col("ban_order") == 0).groupBy("champ").count().orderBy(col("count").desc()).limit(1).collect()
    if ban:
        row = [(league, ban[0]["champ"], ban[0]["count"])]
        league_bans = league_bans.union(spark.createDataFrame(row, schema))

league_bans.show()

+--------------------+----------+-----+
|              league|  champion|count|
+--------------------+----------+-----+
|2011 Season World...|    Rumble|   10|
|2012 Season World...|    Anivia|    7|
|2013 Season World...|       Zed|   20|
|2014 Season World...|   Alistar|   26|
|2015 Season World...| Gangplank|   25|
|2016 Season World...|   Nidalee|   30|
|2017 Season World...|   Kalista|   90|
|2018 Season World...|Tahm Kench|   26|
|2019 Season World...|  Pantheon|  101|
|2020 Season World...|    Lucian|   26|
|2021 Season World...|   LeBlanc|   35|
|2022 Season World...|     Yuumi|   35|
|2023 Season World...|    Maokai|   29|
|2024 Season World...|    Aurora|    6|
|2015 Mid-Season I...|   LeBlanc|   11|
|2016 Mid-Season I...|   Alistar|   15|
|2017 Mid-Season I...|    Kennen|   22|
|2018 Mid-Season I...|    Kai'Sa|   18|
|2019 Mid-Season I...| Jarvan IV|   25|
|2021 Mid-Season I...|  Renekton|   28|
+--------------------+----------+-----+
only showing top 20 rows



Q6: Performance by region in MSI

In [15]:
tournamentresults = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://quocthanh:5432/ETL_Leaguepedia") \
    .option("dbtable", "tournamentresults") \
    .option("user", "quocthanh") \
    .option("password", "quocthanh1804") \
    .load()

teams = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://quocthanh:5432/ETL_Leaguepedia") \
    .option("dbtable", "teams") \
    .option("user", "quocthanh") \
    .option("password", "quocthanh1804") \
    .load()

In [16]:
q6 = tournamentresults.filter((col("overviewpage").contains("Mid-Season Invitational")) & (col("place").isin(["1", "2"]))).withColumn("team", when(col("team") == "DWG KIA", "Dplus KIA").otherwise(col("team"))) \
    .join(teams, col("team") == teams.name).select("team", "place", "overviewpage", "region").sort("overviewpage")
regions = q6.select("region").distinct().toPandas()["region"].tolist()
schema = StructType([
    StructField("region", StringType(), True),
    StructField("1st", StringType(), True),
    StructField("2nd", StringType(), True),
])
performances = spark.createDataFrame([], schema)
for region in regions:
    first = q6.filter((col("region") == region) & (col("place") == "1")).count()
    second = q6.filter((col("region") == region) & (col("place") == "2")).count()
    row = [(region, first, second)]
    performances = performances.union(spark.createDataFrame(row, schema))
performances.sort("1st", ascending=False).show()

+-------------+---+---+
|       region|1st|2nd|
+-------------+---+---+
|        China|  5|  2|
|        Korea|  3|  4|
|         EMEA|  1|  1|
|North America|  0|  2|
+-------------+---+---+



Q7: Performance by region in Worlds

In [62]:
q7 = tournamentresults.filter((col("overviewpage").like("Season % World Championship")) & (col("place").isin(["1", "2"])))
q7 = q7.union(tournamentresults.filter((col("overviewpage").like("% Season World Championship")) & (col("place").isin(["1", "2"])))).withColumn("team", when((col("team") == "DWG KIA") | (col("team") == "DAMWON Gaming"), "Dplus KIA").otherwise(col("team"))).withColumn("team", when(col("team") == "against All authority", "Against All authority").otherwise(col("team")))\
    .withColumn("team", when(col("team") == "Star Horn Royal Club", "Royal Club").otherwise(col("team"))) \
    .withColumn("team", when(col("team") == "KOO Tigers", "ROX Tigers").otherwise(col("team"))) \
    .join(teams, col("team") == teams.name).select("team", "place", "overviewpage", "region").sort("overviewpage")
q7 = q7.withColumn("region", when(col("region") == "Europe", "EMEA").otherwise(col("region")))
#Filter 2024, 2025 at the time
q7 = q7.filter(~col("overviewpage").contains("2024")).filter(~col("overviewpage").contains("2025"))
performances = spark.createDataFrame([], schema)
regions = q7.select("region").distinct().toPandas()["region"].tolist()
for region in regions:
    first = q7.filter((col("region") == region) & (col("place") == "1")).count()
    second = q7.filter((col("region") == region) & (col("place") == "2")).count()
    row = [(region, first, second)]
    performances = performances.union(spark.createDataFrame(row, schema))
performances.sort("1st", ascending=False).show()

    

+------+---+---+
|region|1st|2nd|
+------+---+---+
| Korea|  8|  6|
| China|  3|  4|
|  EMEA|  1|  3|
|   LMS|  1|  0|
+------+---+---+

